## 构建结构化的tensorflow代码

以下内容严重借鉴于这篇博客[传送门](https://danijar.com/structuring-your-tensorflow-models/), 墙裂建议阅读原文.

### 菜鸟级

In [7]:
import tensorflow as tf

一般来说, 我们自己直接用的脚本都比较简单直接, 写几个函数, 按顺序调用就行了. 如下所示面的代码所示, 下面的代码出自一下链接:
[传送门](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/neural_network_raw.py)

```python
""" Neural Network.
A 2-Hidden Layers Fully Connected Neural Network (a.k.a Multilayer Perceptron)
implementation with TensorFlow. This example is using the MNIST database
of handwritten digits (http://yann.lecun.com/exdb/mnist/).
Links:
    [MNIST Dataset](http://yann.lecun.com/exdb/mnist/).
Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
"""

from __future__ import print_function

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

# Parameters
learning_rate = 0.1
num_steps = 500
batch_size = 128
display_step = 100

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}


# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Construct model
logits = neural_net(X)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images,
                                      Y: mnist.test.labels}))
```


以上代码是否觉得十分熟悉? 反正我自己最开始入门的时候就是学习这样自的代码的. 简单易容, 脚本感强烈. 然而, 一开始看别人的论文中实现的一些库就稍微有点蛋疼了

### 向结构化转型中

我们在慢慢地探索过程中, 就知道了将代码结构话还是很有必要的, 代码的复用行会好很多. 所以, 我们可以不妨尝试着将代码成类的形式:

In [4]:
class Model:

    def __init__(self, data, target):
        data_size        = int(data.get_shape()[1])
        target_size      = int(target.get_shape()[1])
        
        weight           = tf.Variable(tf.truncated_normal([data_size, target_size]))
        bias             = tf.Variable(tf.constant(0.1, shape=[target_size]))
        incoming         = tf.matmul(data, weight) + bias
        
        self._prediction = tf.nn.softmax(incoming)
        cross_entropy    = -tf.reduce_sum(target, tf.log(self._prediction))
        self._optimize   = tf.train.RMSPropOptimizer(0.03).minimize(cross_entropy)
        
        mistakes         = tf.not_equal(
                              tf.argmax(target, 1), tf.argmax(self._prediction, 1))
        self._error      = tf.reduce_mean(tf.cast(mistakes, tf.float32))

    @property
    def prediction(self):
        return self._prediction

    @property
    def optimize(self):
        return self._optimize

    @property
    def error(self):
        return self._error


上面的代码就是类的形式了. 我们在初始话的时候, 就将整个网络的构建好, 将其属性定义好, 使用的时候就用调用属性的方法来获取相应的结果就行了.美滋滋.

其中**_`@property`_**是什么意思呢, 这在python中定义为一个装饰器, 涉及到了函数式编程的思想, 在这里的作用呢, 就是将函数的返回值当成了一个属性值. 如下所示, 接下来我还会继续详细地解释一下这个装饰器的内容, 现在知道这里就可以了:    

``` python
network    = Model(images, labels)
prediction = network.prediciton
error      = network.error
```

其实, 上面的代码一看就知道里面还是存在这很多问题的, 最突出的一点就是将所有的定义都放在到了__init__函数中, 一点都不优雅, 每次使用的时候,都得重新定义一遍, 可复用性极低, 和上面的脚本没有太大额区别, 只是披了一个类的外壳而已.

### 对类进行优化

所以, 我们正确的解锁姿势应该是网络的构建, 预测和错误分析都放进各自的函数中:

In [ ]:
class Model:

    def __init__(self, data, target):
        self.data = data
        self.target = target
        self._prediction = None
        self._optimize = None
        self._error = None

    @property
    def prediction(self):
        if not self._prediction:
            data_size = int(self.data.get_shape()[1])
            target_size = int(self.target.get_shape()[1])
            weight = tf.Variable(tf.truncated_normal([data_size, target_size]))
            bias = tf.Variable(tf.constant(0.1, shape=[target_size]))
            incoming = tf.matmul(self.data, weight) + bias
            self._prediction = tf.nn.softmax(incoming)
        return self._prediction

    @property
    def optimize(self):
        if not self._optimize:
            cross_entropy = -tf.reduce_sum(self.target, tf.log(self.prediction))
            optimizer = tf.train.RMSPropOptimizer(0.03)
            self._optimize = optimizer.minimize(cross_entropy)
        return self._optimize

    @property
    def error(self):
        if not self._error:
            mistakes = tf.not_equal(
                tf.argmax(self.target, 1), tf.argmax(self.prediction, 1))
            self._error = tf.reduce_mean(tf.cast(mistakes, tf.float32))
        return self._error

以上的代码, 应该说还是相当优美的了, 注意到, **`self._prediction`**和**`self._optimize`**他们的定义, 初始化的时候, 前面的定义是有下划线的, 后面的函数的定义的时候, 开头是没有下划线的? 这里有什么奥妙麽, 注意结合**`@property`**这一个装饰器的作用?  

可以看到, 我们在正常调用的时候, 应该调用的方式如下:   
``` Python
    prediction = Model.prediction
```

而初始化的时候, 定义*self._prediciton*的作用是使得这些属性的构造过程只在第一次被调用的时候运行一次, 第二次的时候, 就不会再运行那几行代码了, 所以说, 这种形式还是相当又优雅的. 到这里, 我们应该还是挺满意的了. 

但是突然间, 你会发现*prediction*, *optimize* 以及*error*三个函数的构造基本是一毛一样的, 都是先判断, 后赋值. 好像还是有一点重复和冗余的.

### 高级优化

为什么要叫做高级优化呢, 因为这种优化你要付出蛮多的学习成本, 甚至, 你会觉得不用做这些优化都可以了, 上面的冗余自己可以接受. 但是, 我们得看懂被人的代码呀, 所以我们还是不妨先看一看吧.   
这里用到的关键技术就是`python`里面美丽的装饰器. 装饰器的内容详细解说, 我又要给大家强力安利[**Python cookbook 3rd**](http://python3-cookbook.readthedocs.io/zh_CN/latest/chapters/p09_meta_programming.html). 这里面一个最重要的思想就是函数式编程的思想, 就是函数成为了一等公民, 函数名可以直接用来当做参数进行传递. 就像C语言中的函数指针指向函数定义的首地址, 可以用来传递一样. 函数式编程的另外一个重要的体现就是lambda表达式, 在此提一下不进行分析.

下面我们先来看一个Python装饰器的简单应用:

In [22]:
import time
from functools import wraps

def timethis(func):
    '''
    Decorator that reports the execution time.
    '''
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

In [ ]:
上面定义了一个简单的装饰器, 这个装饰器可以将被修饰的函数的运行时间打印出来.

In [28]:
@timethis
def countdown(n):
     '''
     Counts down
     '''
     while n > 0:
         n -= 1

countdown(100000)
countdown(10000000)


countdown 0.0062198638916015625
countdown 0.595980167388916


跟像下面这样写其实效果是一样的：

顺便说一下，内置的装饰器比如 @staticmethod, @classmethod,@property 原理也是一样的。 

In [29]:
def countdown(n):
     '''
     Counts down
     '''
     while n > 0:
         n -= 1

countdown = timethis(countdown)
countdown(100000)
countdown(10000000)

countdown 0.006100893020629883
countdown 0.5906565189361572


所以, 从上面我们就可以看到, 装饰器其实就是接受一个函数作为输入参数, 然后给调用这返回一个新的被重新构造过的函数, 来给函数添加一些通用的功能. 所以, 我们就可以利用装饰器来简化以上, 我们上面的Tensorflow代码了.

先定义一个装饰器:

In [23]:
import functools

def lazy_property(function):
    attribute = '_cache_' + function.__name__

    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
            setattr(self, attribute, function(self))
        return getattr(self, attribute)

    return decorator

然后再重新改写class Model的代码:

In [26]:
class Model:

    def __init__(self, data, target):
        self.data = data
        self.target = target
        self.prediction
        self.optimize
        self.error

    @lazy_property
    def prediction(self):
        data_size = int(self.data.get_shape()[1])
        target_size = int(self.target.get_shape()[1])
        weight = tf.Variable(tf.truncated_normal([data_size, target_size]))
        bias = tf.Variable(tf.constant(0.1, shape=[target_size]))
        incoming = tf.matmul(self.data, weight) + bias
        return tf.nn.softmax(incoming)

    @lazy_property
    def optimize(self):
        cross_entropy = -tf.reduce_sum(self.target, tf.log(self.prediction))
        optimizer = tf.train.RMSPropOptimizer(0.03)
        return optimizer.minimize(cross_entropy)

    @lazy_property
    def error(self):
        mistakes = tf.not_equal(
            tf.argmax(self.target, 1), tf.argmax(self.prediction, 1))
        return tf.reduce_mean(tf.cast(mistakes, tf.float32))

整个世界都变得简单和明了了. 上面的惰性检查就全部被一个装饰器来替代了.

### 再煮一个栗子

利用装饰器来给我们的函数添加name_scope, 装饰器定义如下, 其具体的用法跟上面的基本一毛一样.

In [ ]:
import functools

def define_scope(function):
    attribute = '_cache_' + function.__name__

    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
            with tf.variable_scope(function.__name):
                setattr(self, attribute, function(self))
        return getattr(self, attribute)

    return decorator

### 总结

上面主要介绍了使用类来编写tensorflow模型的一个基本的模板, 然后在使用装饰器来对这个类进行优化, 既提高了代码的可读性, 又提高了代码的运行效率.   
细心的你, 应该也可以发现, 其实上面的编写和优化和tensorflow本身并没有很强的相关性, 所以其实这个模板也完全可以套用到你喜欢的各种深度学习框架中, 以上希望能够给你带来一点参考.